In [10]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Add the project root directory to the Python path
# This assumes the notebook is in a subdirectory of the project root
project_root = os.path.dirname(current_dir)
if project_root not in sys.path:
    sys.path.append(project_root)

print(f"Project root directory '{project_root}' added to Python path")
print(f"Current Python path: {sys.path}")


Project root directory 'd:\workspace\docent' added to Python path
Current Python path: ['C:\\Users\\bigbl\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'C:\\Users\\bigbl\\AppData\\Local\\Programs\\Python\\Python312\\DLLs', 'C:\\Users\\bigbl\\AppData\\Local\\Programs\\Python\\Python312\\Lib', 'C:\\Users\\bigbl\\AppData\\Local\\Programs\\Python\\Python312', 'd:\\workspace\\docent\\venv_dcnt', '', 'd:\\workspace\\docent\\venv_dcnt\\Lib\\site-packages', 'd:\\workspace\\docent\\venv_dcnt\\Lib\\site-packages\\win32', 'd:\\workspace\\docent\\venv_dcnt\\Lib\\site-packages\\win32\\lib', 'd:\\workspace\\docent\\venv_dcnt\\Lib\\site-packages\\Pythonwin', 'd:\\workspace\\docent']


In [21]:
import utils
from IPython.display import display
# utils.display_image() 함수는 이미지를 반환만 하고 표시하지 않습니다
# display() 함수를 사용하여 이미지를 직접 표시합니다
image_path = os.path.join("relics", "348", "bon001958-00-01.jpg")
#display(utils.Image(image_path))


In [71]:
#os.path.join("relics", "348", "bon001958-00-01.jpg")
import anthropic
import base64

# 로컬 파일을 base64로 인코딩
def get_base64_data(file_path):
    with open(file_path, "rb") as f:
        base64_data = base64.standard_b64encode(f.read()).decode("utf-8")
    return base64_data

In [72]:
#os.path.join("relics", "348", "bon001958-00-01.jpg")
import anthropic

client = anthropic.Anthropic()

def get_response(img_path, input_message):
    message = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=1024,
        temperature=0,
        stop_sequences=["</json>"],
        messages=[
            {   
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": get_base64_data(img_path)
                        }
                    },
                    {
                        "type": "text", 
                        "text": input_message
                    }
                ]
            },
            {"role": "assistant", "content": "<json>"}
        ],
    )
    return message

In [73]:
import json
json_path = "combined_relics.json"

with open(json_path, "r", encoding="utf-8") as f:
    combined_data = json.load(f)  

In [74]:
def read_json(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        combined_data = json.load(f)  
    return combined_data


In [94]:
numbers = [348, 1431, 3166, 4390, 178434, 179148, 36522635, 36548915, 36551987, 36559575]

# Filter combined_data to only include items with keys matching the numbers
filtered_data = {str(num): combined_data.get(str(num)) for num in numbers if str(num) in combined_data}

filtered_data

print(len(filtered_data))
#filtered_data

10


In [ ]:
# {{"저의 느낌입니다.":<5문장 이내로>, "잘 어울리는 MBTI":<MBTI>, "MBIT 선정 이유":<MBTI 선정 이유>, "이런 감정 상태인 사람에게 추천하고 싶어요":<5문장 이내로>, "지금까지 말한 감상을 3개의 형용사로 표현해":[...]}}

user_message = """
당신은 미적 감각이 예리하고 정서적으로 민감한 한국의 국립중앙박물관 도슨트입니다. 당신의 의견을 반영해 전시품 감상 데이터베이스를 구축하려합니다.
유물의 세부적이 내용은 박물관에서 제공하는 <information>를 참고하세요.

<information>
{information}
</information>

<instructions>
1. <json> 태그로 감싸 다음 JSON 포맷으로 응답하세요('과거와 현재를 연결하는 시간' 등의 표현은 하지 말아주세요):
{{"저의 느낌입니다.":<5문장 이내로>, "이런 상태에 놓인 사람에게 추천하고 싶어요":<구체적인 상황을 5문장 이내로 묘사>, "작품의 첫인상을 형용사로 3순위까지 표현해":[<1순위>,<2순위>,<3순위>]}}

</instructions>
"""

response_json = {}

for idx, (key, value) in enumerate(filtered_data.items()):
    if idx > 2: 
        break
    print(key)
    img_path = os.path.join("relics", key, value['img'].split('/')[-1])
    info_json_path = os.path.join("relics", key, "info.json")
    #print(info_json_path)
    info = read_json(info_json_path)
    information = str({"설명": info['content'], "세부정보": info['info']})
    #print(information)    
    response = get_response(img_path, user_message.format(information=information))
    response_json[key] = json.loads(response.content[0].text)
    
# Save the response_json to a file
output_path = "response.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(response_json, f, ensure_ascii=False, indent=4)

348
1431
3166


In [112]:
response_json

{'348': {'저의 느낌입니다.': '섬세한 조각 기법으로 표현된 미륵보살의 우아한 자태가 깊은 평온함을 전해줍니다. 얼굴의 미소와 풍만한 신체의 곡선이 조화롭게 어우러져 부드러운 아름다움을 느끼게 합니다. 화려한 장식과 얇은 법의가 만들어내는 대비가 신성함과 인간적 친근함을 동시에 담고 있어요.',
  '이런 상태에 놓인 사람에게 추천하고 싶어요': '인생의 중요한 결정 앞에서 망설이고 있는 사람에게 이 미륵보살상을 추천합니다. 복잡한 감정으로 마음의 평화를 찾지 못하는 이에게 보살의 온화한 미소가 위안을 줄 것입니다. 누군가를 위해 헌신하고 싶은 마음이 있지만 방법을 찾지 못하는 사람에게 김지성의 효심이 담긴 이 작품이 영감을 줄 수 있을 것입니다.',
  '작품의 첫인상 형용사로 3순위까지 표현해': ['우아한', '평온한', '장엄한']},
 '1431': {'저의 느낌입니다.': "청자의 비취색 유약이 은은하게 빛나며 신비로운 분위기를 자아냅니다. 거북이와 용이 결합된 독특한 형태가 상상력을 자극합니다. 등껍질에 새겨진 '왕'자는 고귀함과 권위를 상징하며 작품에 깊이를 더합니다. 정교한 세부 표현에서 고려 장인의 뛰어난 기술과 미적 감각이 느껴집니다.",
  '이런 상태에 놓인 사람에게 추천하고 싶어요': "일상에 지쳐 상상력과 영감이 필요한 예술가나 창작자에게 추천합니다. 전통과 현대를 연결하는 새로운 아이디어를 찾는 디자이너에게 영감을 줄 것입니다. 권위와 책임감에 대해 고민하는 리더에게 '왕'자의 의미를 생각해볼 기회가 될 것입니다. 동양 미학과 상징성에 관심 있는 문화 연구자에게도 깊은 통찰을 제공할 것입니다.",
  '작품의 첫인상 형용사로 3순위까지 표현해': ['신비로운', '정교한', '고귀한']},
 '3166': {'저의 느낌입니다.': '이 묘법연화경의 낡은 표지에서 세월의 흔적이 고스란히 느껴집니다. 세조 시대 한글 번역본이라는 점에서 우리 문화유산의 소중함이 더욱 와닿습니다. 책의 황토색 표지와 세로로 적힌 제목이 고전적인 

In [ ]:
from pprint import pprint
pprint(json.loads(response.content[0].text))